# Particionado en Spark

## Requisitos Previos

Instalar Spark y Java en la Máquina Virtual (VM)

In [1]:
# instalar Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# descargar spark 3.5.0
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [2]:
ls -l # verificar que el .tgz está ahí

total 391016
drwxr-xr-x 1 root root      4096 Nov 17 14:29 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [3]:
# descomprimirlo
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:

!pip install py4j

# Para maps
!pip install folium
!pip install plotly

Definir el entorno

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Iniciar Sesión de Spark (Spark Session)

---

In [7]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# crear la sesión
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [8]:
spark

In [9]:
# Para optimización de conversión a Pandas
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
# Importar funciones sql
from pyspark.sql.functions import *

Descargar y subir los conjuntos de datos (datasets) (EMPLOYEES.CSV)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving employees.csv to employees.csv


## Examples

### Patitioning

In [12]:
# Cargar el csv como un DF
trabajadoresDF = spark.read.option("inferSchema", "true").option("header", "true").csv("employees.csv")

In [13]:
# Muestra como esta actualmente particionado el DF
trabajadoresDF \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0|30003|
+-----------+-----+



In [14]:
# Ahora vamos a hacer que el DF se particione en 20 partes
trabajadoresRepDF = trabajadoresDF.repartition(20)
trabajadoresRepDF \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("partitionId")) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 1500|
|          1| 1500|
|          2| 1500|
|          3| 1500|
|          4| 1500|
|          5| 1500|
|          6| 1500|
|          7| 1500|
|          8| 1500|
|          9| 1501|
|         10| 1501|
|         11| 1501|
|         12| 1500|
|         13| 1500|
|         14| 1500|
|         15| 1500|
|         16| 1500|
|         17| 1500|
|         18| 1500|
|         19| 1500|
+-----------+-----+



In [15]:
# Ahora podemos usar coalesce para reducir a 5 por ejemplo el numero de particiones y que no sean tantas
trabajadoresRepDF \
  .coalesce(5) \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("partitionId")) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 6000|
|          1| 6001|
|          2| 6002|
|          3| 6000|
|          4| 6000|
+-----------+-----+



## Ejercicios

1. Prueba como funciona el partition y el coalesce por tu parte con cualquier dataset de los disponibles